In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold,GridSearchCV,RandomizedSearchCV,cross_val_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler,Normalizer,RobustScaler
from sklearn.preprocessing import LabelEncoder

In [21]:
train = pd.read_csv('D:\Capstone Project\Train\Train.csv')
sub = pd.read_csv('D:\Capstone Project\Train\sample_submmission.csv')

camp1 = pd.read_csv('D:\Capstone Project\Train\First_Health_Camp_Attended.csv')
camp2= pd.read_csv('D:\Capstone Project\Train\Second_Health_Camp_Attended.csv')
camp3 = pd.read_csv('D:\Capstone Project\Train\Third_Health_Camp_Attended.csv')
patient_profile = pd.read_csv('D:\Capstone Project\Train\Patient_Profile.csv')
camp_detail = pd.read_csv('D:\Capstone Project\Train\Health_Camp_Detail.csv')
test = pd.read_csv('D:\Test_ha.csv')

In [3]:
train.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,489652,6578,10-Sep-05,4,0,0,0,2
1,507246,6578,18-Aug-05,45,5,0,0,7
2,523729,6534,29-Apr-06,0,0,0,0,0
3,524931,6535,07-Feb-04,0,0,0,0,0
4,521364,6529,28-Feb-06,15,1,0,0,7


In [4]:
test.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,505701,6548,21-May-06,1,0,0,0,2
1,500633,6584,2-Jun-06,0,0,0,0,0
2,506945,6582,10-Aug-06,0,0,0,0,0
3,497447,6551,27-Aug-06,0,0,0,0,0
4,496446,6533,19-Sep-06,0,0,0,0,0


In [5]:

camp1.head()

,Patient_ID,Health_Camp_ID,Donation,Health_Score,Unnamed: 4
0,506181,6560,40,0.439024,NaN
1,494977,6560,20,0.097561,NaN
2,518680,6560,10,0.048780,NaN
3,509916,6560,30,0.634146,NaN
4,488006,6560,20,0.024390,NaN


In [22]:
camp1.drop(columns=['Unnamed: 4'],axis=1,inplace=True)

In [7]:
camp1.head()

,Patient_ID,Health_Camp_ID,Donation,Health_Score
0,506181,6560,40,0.439024
1,494977,6560,20,0.097561
2,518680,6560,10,0.048780
3,509916,6560,30,0.634146
4,488006,6560,20,0.024390


In [8]:
camp2.head()

,Patient_ID,Health_Camp_ID,Health Score
0,526631,6536,0.875136
1,509122,6536,0.755700
2,498864,6536,0.673181
3,515398,6536,0.722041
4,504624,6536,0.464712


In [9]:
camp3.head()

,Patient_ID,Health_Camp_ID,Number_of_stall_visited,Last_Stall_Visited_Number
0,517875,6527,3,1
1,504692,6578,1,1
2,504692,6527,3,1
3,493167,6527,4,4
4,510954,6528,2,2


In [12]:
camp3['Number_of_stall_visited'].value_counts()

1    1822
2    1243
3    1145
5    1034
4     701
6     536
0      18
7      16
Name: Number_of_stall_visited, dtype: int64

In [23]:

camp3=camp3[camp3.Number_of_stall_visited>0]
camp3.Number_of_stall_visited.value_counts()

1    1822
2    1243
3    1145
5    1034
4     701
6     536
7      16
Name: Number_of_stall_visited, dtype: int64

In [24]:

all_camps=pd.concat([camp1,camp2,camp3],sort=False)

all_camps=all_camps.fillna(0)

In [25]:
all_camps.head()

,Patient_ID,Health_Camp_ID,Donation,Health_Score,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number
0,506181,6560,40.0,0.439024,0.0,0.0,0.0
1,494977,6560,20.0,0.097561,0.0,0.0,0.0
2,518680,6560,10.0,0.048780,0.0,0.0,0.0
3,509916,6560,30.0,0.634146,0.0,0.0,0.0
4,488006,6560,20.0,0.024390,0.0,0.0,0.0


In [28]:
all_camps['outcome']=1

In [26]:

train_merge = pd.merge(train, patient_profile, on='Patient_ID', how='left')
test_merge = pd.merge(test, patient_profile, on='Patient_ID', how='left')

In [27]:
train_merge['Registration_Date']=train_merge['Registration_Date'].fillna(train_merge['First_Interaction'])

In [29]:
train_merge= pd.merge(train_merge,camp_detail,on=['Health_Camp_ID'], how='left')
test_merge = pd.merge(test_merge,camp_detail,on=['Health_Camp_ID'], how='left')

In [30]:

train_merge = pd.merge(train, patient_profile, on='Patient_ID', how='left')
test_merge = pd.merge(test, patient_profile, on='Patient_ID', how='left')

In [31]:
train_merge= pd.merge(train_merge,all_camps,on=['Patient_ID', 'Health_Camp_ID'], how='left')
test_merge = pd.merge(test_merge,all_camps,on=['Patient_ID', 'Health_Camp_ID'], how='left')
train_merge['outcome'] = train_merge['outcome'].fillna(0).astype('int')
test_merge.drop(columns=['outcome'],axis=1,inplace=True)

In [32]:
train_merge.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Online_Follower,LinkedIn_Shared,...,Age,First_Interaction,City_Type,Employer_Category,Donation,Health_Score,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number,outcome
0,489652,6578,10-Sep-05,4,0,0,0,2,0,0,...,None,06-Dec-04,NaN,NaN,0.0,0.0,0.000000,2.0,1.0,1
1,507246,6578,18-Aug-05,45,5,0,0,7,0,0,...,40,08-Sep-04,C,Others,NaN,NaN,NaN,NaN,NaN,0
2,523729,6534,29-Apr-06,0,0,0,0,0,0,0,...,None,22-Jun-04,NaN,NaN,0.0,0.0,0.402054,0.0,0.0,1
3,524931,6535,07-Feb-04,0,0,0,0,0,0,0,...,None,07-Feb-04,I,NaN,NaN,NaN,NaN,NaN,NaN,0
4,521364,6529,28-Feb-06,15,1,0,0,7,0,0,...,40,04-Jul-03,I,Technology,0.0,0.0,0.845597,0.0,0.0,1


In [33]:
train_merge.shape

(75278, 24)